In [334]:
import pandas as pd
import json
import glob
import itertools

def dedupe(k):
    k.sort()
    return list(k for k,_ in itertools.groupby(k))

In [377]:
df = []
for fp in glob.glob("../*.json"):
    with open(fp, 'r') as file:
        d = json.load(file)
        df.append(pd.json_normalize(d))

In [378]:
df = pd.concat(df)
df['correct'] = df["true"] == df["prediction.clue"]

In [379]:
bad_tasks = [-211950127084943975, 2704333446108177290, -2291809755259426270, 1038356832316350171, 
             -8850885074809592414, 8113065641356338218, 327989699109363352, 8795692948402043846, -4473970270804012794,
            2664444452045579339, -5412451596820381002, -424210961467812871, 2680223075587281955,
            8370818600640760487, -5838975061206382918, -7174004820279856653, -2827548842012971459, 699748936081512751,
             950962217963898153]

In [382]:
hash_l_map = {hash(tuple(l)):l for l in dedupe(df.words_to_hit.values.tolist())}
df['task_hash'] = df.words_to_hit.apply(lambda x: hash(tuple(x)))

In [383]:
df = df[~df['task_hash'].isin(bad_tasks)]

In [384]:
model_groupers = ['model_name', 'threshold', 'algorithm']
precision_groupers = ['model_name', 'threshold', 'algorithm', 'task_hash']

In [385]:
df['rank'] = df.groupby(precision_groupers).cumcount()

/var/folders/05/qwnb9n592gndwnbn3qr4l40h0000gn/T/ipykernel_1815/203688535.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rank'] = df.groupby(precision_groupers).cumcount()


In [386]:
found_at_20 = df.groupby(precision_groupers).apply(lambda x: 1 if sum(x["correct"] > 0) else 0).reset_index(name='found@20')
df = pd.merge(df, found_at_20, how='left', on=precision_groupers)

In [387]:
found_at_10 = df[df['rank']<11].groupby(precision_groupers).apply(lambda x: 1 if sum(x["correct"] > 0) else 0).reset_index(name='found@10')
df = pd.merge(df, found_at_10, how='left', on=precision_groupers)

In [388]:
found_at_5 = df[df['rank']<6].groupby(precision_groupers).apply(lambda x: 1 if sum(x["correct"] > 0) else 0).reset_index(name='found@5')
df = pd.merge(df, found_at_5, how='left', on=precision_groupers)

In [391]:
df.groupby(['model_name', 'threshold', 'algorithm'])["found@5"].mean().sort_values().tail(20)

model_name  threshold  algorithm             
fasttext    0.500000   MeanIndividualDistance    0.050633
            0.428571   SummedNearestNeighbour    0.050633
                       MeanIndividualDistance    0.050633
            0.285714   SummedNearestNeighbour    0.050633
paragram    0.071429   SummedNearestNeighbour    0.050633
fasttext    0.500000   SummedNearestNeighbour    0.050633
paragram    0.142857   SummedNearestNeighbour    0.050633
            0.214286   SummedNearestNeighbour    0.063291
fasttext    0.071429   MeanIndividualDistance    0.075949
            0.285714   MeanIndividualDistance    0.075949
            0.000000   MeanIndividualDistance    0.075949
            0.214286   MeanIndividualDistance    0.075949
            0.142857   MeanIndividualDistance    0.075949
            0.357143   SummedNearestNeighbour    0.088608
glove       0.285714   SummedNearestNeighbour    0.088608
fasttext    0.357143   MeanIndividualDistance    0.088608
glove       0.214286   Sum

In [392]:
winner = df[(df['model_name']=='glove')&((df['threshold']>0.13)&(df['threshold']<0.15))&(df['algorithm']=='SummedNearestNeighbour')]

In [396]:
winner[(winner["found@5"]==1)&(winner['rank']<6)].head(5)

,model_name,threshold,algorithm,words_to_hit,true,prediction.clue,prediction.similarity,prediction.linked_words,prediction.score,prediction.num_words_linked,correct,task_hash,rank,found@20,found@10,found@5
8300,glove,0.142857,SummedNearestNeighbour,"[alps, europe]",france,pyrenees,0.550288,"[alps, europe]",0.778225,2,False,-8730468947444872091,0,1,1,1
8301,glove,0.142857,SummedNearestNeighbour,"[alps, europe]",france,scandinavia,0.547306,"[alps, europe]",0.774008,2,False,-8730468947444872091,1,1,1,1
8302,glove,0.142857,SummedNearestNeighbour,"[alps, europe]",france,france,0.525593,"[alps, europe]",0.743300,2,True,-8730468947444872091,2,1,1,1
8303,glove,0.142857,SummedNearestNeighbour,"[alps, europe]",france,italy,0.519630,"[alps, europe]",0.734867,2,False,-8730468947444872091,3,1,1,1
8304,glove,0.142857,SummedNearestNeighbour,"[alps, europe]",france,asia,0.504477,"[alps, europe]",0.713439,2,False,-8730468947444872091,4,1,1,1
